# Testing Notebook

## Start container mssqlsrv1

Use docker

In [133]:
# -- Start mssqlsrv1
$null = docker start mssqlsrv1
docker ps

CONTAINER ID   IMAGE               COMMAND                  CREATED       STATUS       PORTS                                           NAMES
30ea66275f78   mongo               "docker-entrypoint.s…"   2 weeks ago   Up 2 hours   0.0.0.0:27017->27017/tcp, :::27017->27017/tcp   mongodb1
e7ff4a53d0dd   mssql:2019-latest   "/opt/mssql/bin/perm…"   2 weeks ago   Up 3 hours   0.0.0.0:8433->1433/tcp, :::8433->1433/tcp       mssqlsrv1


## Create a TestingDB on MSSQL Server

Use dbatools, what else?

In [150]:
# -- Create new database 'TestingDB'
$database = 'TestingDB'
if(-not(Get-InstalledModule -Name dbatools)){
    Install-Module -Name dbatools
}
if(-not(Get-Module -Name dbatools)){
    Import-Module -Name dbatools
}
if([String]::IsNullOrEmpty($sqlcred)){
    $secStringPassword = ConvertTo-SecureString 'yourStrong(!)Password' -AsPlainText -Force
    $sqlcred = New-Object System.Management.Automation.PSCredential ('sa', $secStringPassword)
}
$mssqlsrv = Connect-DbaInstance -SqlInstance 'localhost:8433' -SqlCredential $sqlcred
$dbexists = Get-DbaDatabase -SqlInstance $mssqlsrv  -Database $database
if($dbexists){
    "$database already exists"
}else{
    New-DbaDatabase -SqlInstance $mssqlsrv -Name $database
}

TestingDB already exists


## Create new Testing Table

Use dbatools, what else?

In [135]:
# -- Create new table 'Test_Case1'
$table    = 'Test_Case1'
$database = 'TestingDB'
$Columns  =  @()
$Columns  += @{
    Name      = 'ID'
    Type      = 'int'
    Identity  = $true
}
$Columns  += @{
    Name      = 'TimeStamp'
    Type      = 'datetime'
    Nullable  =  $false
    Default   =  'GETDATE()'
}
$Columns  += @{
    Name      = 'Testcase'
    Type      = 'Nvarchar'
    MaxLength = 4000
    Nullable  =  $true
}
$Columns  += @{
    Name      = 'HowToTest'
    Type      = 'Nvarchar'
    MaxLength = 4000
    Nullable  =  $true
}
$Columns  += @{
    Name      = 'ExpectetResult'
    Type      = 'Nvarchar'
    MaxLength = 4000
    Nullable  =  $true
}
$Columns  += @{
    Name      = 'ReceivedResult'
    Type      = 'Nvarchar'
    MaxLength = 4000
    Nullable  =  $true
}
$Columns  += @{
    Name      = 'Status'
    Type      = 'Nvarchar'
    MaxLength = 10
    Nullable  =  $true
}
$tblexists = Get-DbaDbTable -SqlInstance $mssqlsrv -Database $database -Table $table
if($tblexists){
    "$tblexists already exists"
}else{
    New-DbaDbTable -SqlInstance $mssqlsrv -Database $database -Name $table -ColumnMap $Columns | Select SqlInstance, Database, Name
}


SqlInstance Database  Name
----------- --------  ----
mssqlsrv1   TestingDB Test_Case1



## Use Cases

In [155]:
# -- Connect to MSSQL Server
$database = 'TestingDB'
if(-not(Get-InstalledModule -Name dbatools)){
    Install-Module -Name dbatools
}
if(-not(Get-Module -Name dbatools)){
    Import-Module -Name dbatools
}
if([String]::IsNullOrEmpty($mssqlsrv)){
    if([String]::IsNullOrEmpty($sqlcred)){
        $secStringPassword = ConvertTo-SecureString 'yourStrong(!)Password' -AsPlainText -Force
        $sqlcred = New-Object System.Management.Automation.PSCredential ('sa', $secStringPassword)
    }
    $mssqlsrv = Connect-DbaInstance -SqlInstance 'localhost:8433' -SqlCredential $sqlcred
}else{
    "already connected to $database"
}


ComputerName Name           Product              Version   HostPlatform IsAzure
------------ ----           -------              -------   ------------ -------
localhost    localhost,8433 Microsoft SQL Server 15.0.4123 Linux        False  



In [137]:
# -- Start containers
$container = 'mongodb1'
$result = docker ps -a --filter "name=$container"
if([String]::IsNullOrEmpty($result)){
    docker start $container
}else{
    "$container already running"
}

mongodb1 already running


### Use Case 1

In [138]:
# -- Test 1 docker image
$image = 'mongo'
$result = docker image ls $image 
$string = $result[1].Trim() -split '\s+'

$Testcase       = "Test is Image $image on local docker repository"
$HowToTest      = "docker image ls $image"
$ExpectetResult = "Repository, Tag, Image ID, Date created, Size of the Image"
$ReceivedResult = $string -join ", "
$PassFail       = if($ReceivedResult -match $image){'Pass'}else{'Fail'}
$PassFail

# -- INSERT INTO [TestingDB].[dbo].[Test_Case1]
$SqlQuery   = @"
INSERT INTO [$database].[dbo].[$table] (
    TimeStamp,Testcase,HowToTest,ExpectetResult,ReceivedResult,Status
)
VALUES (
    CURRENT_TIMESTAMP,'$($Testcase)','$($HowtoTest)','$($ExpectetResult)','$($ReceivedResult)','$($PassFail)'
)
"@  
Invoke-DbaQuery -SqlInstance $mssqlsrv -Database $database -Query $SqlQuery -verbose

Pass


In [139]:
# -- Test 2 docker container
$container = 'mongodb1'
$result = docker ps -a --filter "name=$container"
$string = $result[1].Trim() -split '\s+'

$Testcase       = "Test is container $container on local docker repository"
$HowToTest      = "docker ps -a --filter ""name=$container"""
$ExpectetResult = "Container ID, Image, Command, Date created, Status, Ports, Names"
$ReceivedResult = $string -join ", "
$PassFail       = if($ReceivedResult -match $container){'Pass'}else{'Fail'}
$PassFail

# -- INSERT INTO [TestingDB].[dbo].[Test_Case1]
$SqlQuery   = @"
INSERT INTO [$database].[dbo].[$table] (
    TimeStamp,Testcase,HowToTest,ExpectetResult,ReceivedResult,Status
)
VALUES (
    CURRENT_TIMESTAMP,'$($Testcase)','$($HowtoTest)','$($ExpectetResult)','$($ReceivedResult)','$($PassFail)'
)
"@  
Invoke-DbaQuery -SqlInstance $mssqlsrv -Database $database -Query $SqlQuery -verbose

Pass


In [140]:
# -- Test 3 mongodb access
$container = 'mongodb1'
$ports = docker inspect --format='{{.NetworkSettings.Ports}}' $container
$result = Invoke-WebRequest -Uri 'http://localhost:27017'

$Testcase       = "Test is $container is accessible over http"
$HowToTest      = "Invoke-WebRequest -Uri ""http://localhost:27017"""
$ExpectetResult = "StatusCode 200, StatusDescription OK"
$ReceivedResult = "$($result.StatusCode), $($result.StatusDescription), $($result.Content)"
$PassFail       = if($result.StatusCode -eq 200){'Pass'}else{'Fail'}
$PassFail

# -- INSERT INTO [TestingDB].[dbo].[Test_Case1]
$SqlQuery   = @"
INSERT INTO [$database].[dbo].[$table] (
    TimeStamp,Testcase,HowToTest,ExpectetResult,ReceivedResult,Status
)
VALUES (
    CURRENT_TIMESTAMP,'$($Testcase)','$($HowtoTest)','$($ExpectetResult)','$($ReceivedResult)','$($PassFail)'
)
"@  
Invoke-DbaQuery -SqlInstance $mssqlsrv -Database $database -Query $SqlQuery -verbose

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                Pass


In [141]:
# -- Test 4 mongodb database
if(-not(Get-InstalledModule -Name Mdbc)){
    Install-Module -Name Mdbc
}
if(-not(Get-Module -Name Mdbc)){
    Import-Module -Name Mdbc
}

$mongodb = 'admin'
$mongo_databases = Connect-Mdbc mongodb://localhost:27017 | Get-MdbcDatabase
$result = $mongo_databases.DatabaseNamespace
$dbs = $null
foreach($item in $result.DatabaseName){
    $dbs = $item + ", " + $dbs
}

$Testcase       = "Test is $mongodb database on $container is available"
$HowToTest      = "Connect-Mdbc mongodb://localhost:27017 | Get-MdbcDatabase"
$ExpectetResult = "DatabaseNamespace $mongodb"
$ReceivedResult = $dbs.TrimEnd(', ')
$PassFail       = if($result -contains $mongodb){'Pass'}else{'Fail'}
$PassFail

# -- INSERT INTO [TestingDB].[dbo].[Test_Case1]
$SqlQuery   = @"
INSERT INTO [$database].[dbo].[$table] (
    TimeStamp,Testcase,HowToTest,ExpectetResult,ReceivedResult,Status
)
VALUES (
    CURRENT_TIMESTAMP,'$($Testcase)','$($HowtoTest)','$($ExpectetResult)','$($ReceivedResult)','$($PassFail)'
)
"@  
Invoke-DbaQuery -SqlInstance $mssqlsrv -Database $database -Query $SqlQuery -verbose

Pass


In [148]:
# -- SELECT * FROM [TestingDB].[dbo].[Test_Case1]
$SqlQuery  = "SELECT * FROM [$database].[dbo].[$table]"
Invoke-DbaQuery -SqlInstance $mssqlsrv -Database $DBname -Query $SqlQuery | Select ID,Testcase,Status


ID Testcase                                              Status
-- --------                                              ------
 1 Test is Image mongo on local docker repository        Pass
 2 Test is container mongodb1 on local docker repository Pass
 3 Test is mongodb1 is accessible over http              Pass
 4 Test is admin database on mongodb1 is available       Pass



In [142]:
$container = @('mongodb1')
foreach($item in $container){
    [PSCustomObject]@{
        Hostname  = docker inspect --format='{{.Config.Hostname}}' $item 
        IPAddress = docker inspect --format='{{.NetworkSettings.Networks.custom.IPAddress}}' $item 
        Path      = docker inspect --format='{{.Path}}' $item 
        Ports     = docker inspect --format='{{.NetworkSettings.Ports}}' $item 
        ExposedPorts = docker inspect --format='{{.Config.ExposedPorts}}' $item 
    }
}
#docker inspect mongodb1


Hostname     : mongodb1
IPAddress    : 172.18.0.3
Path         : docker-entrypoint.sh
Ports        : map[27017/tcp:[{0.0.0.0 27017} {:: 27017}]]
ExposedPorts : map[27017/tcp:{}]




In [160]:
$database = 'TestingDB'
if([String]::IsNullOrEmpty($sqlcred)){
    $secStringPassword = ConvertTo-SecureString 'yourStrong(!)Password' -AsPlainText -Force
    $sqlcred = New-Object System.Management.Automation.PSCredential ('sa', $secStringPassword)
}
$mssqlsrv = Connect-DbaInstance -SqlInstance 'localhost:8433' -SqlCredential $sqlcred -Database $database

Import-DbaCsv -Path /Users/Tinu/Downloads/Test_MongoDB.csv -SqlInstance $mssqlsrv -Database $database -NullValue "NULL" -AutoCreateTable -verbose

VERBOSE: [13:06:00][Import-DbaCsv] Started at 05/27/2021 13:06:00
VERBOSE: [13:06:00][Import-DbaCsv] Table name not specified, using Test_MongoDB
VERBOSE: [13:06:00][Import-DbaCsv] Schema not specified, using dbo
VERBOSE: Performing the operation "Starting transaction in TestingDB" on target "localhost,8433".
VERBOSE: [13:06:00][Import-DbaCsv] Table does not exist
VERBOSE: Performing the operation "Creating table Test_MongoDB" on target "localhost,8433".


VERBOSE: [13:06:00][New-SqlTable] Successfully created table dbo.Test_MongoDB with the following column definitions:
 [ID] varchar(MAX)
 [TimeStamp] varchar(MAX)
 [Testcase] varchar(MAX)
 [HowToTest] varchar(MAX)
 [ExpectetResult] varchar(MAX)
 [ReceivedResult] varchar(MAX)
 [Status] varchar(MAX)
VERBOSE: [13:06:00][New-SqlTable] This is inefficient but allows the script to import without issues.
VERBOSE: [13:06:00][New-SqlTable] Consider creating the table first using best practices if the data will be used in production.
VERBOSE: [13:06:00][Import-DbaCsv] Starting bulk copy for Test_MongoDB.csv
VERBOSE: Performing the operation "Performing import from /Users/Tinu/Downloads/Test_MongoDB.csv" on target "localhost,8433".
VERBOSE: Performing the operation "Finalizing import" on target "localhost,8433".
VERBOSE: [13:06:00][Import-DbaCsv] 4 total rows copied

ComputerName  : localhost
InstanceName  : MSSQLSERVER
SqlInstance   : mssqlsrv1
Database      : TestingDB
Table         : Test_Mongo